## compute model derived fields (vorticity, steric height)

This notebook can be used to compute steric height and vorticity for regional llc4320 output.

Output is stored in new netCDF files in a "derived" directory.

Requires that an Argo climatology has been downloaded and stored locally.

NOTE: Suggested changes:
- computing strain, MLD, or other parameters
- making code work with other simulation data
- turned this notebook into a script that is called by the parent notebook
- loading the reference T/S data from ERDDAP (or another gridded Argo database)

Initial commit Aug 9, 2021 by kdrushka

In [21]:
#pip install numba

In [22]:
#pip install --upgrade dask[distributed]

In [23]:
%matplotlib inline
import os
# import sys
# import fsspec
import numpy as np
import glob
# import re
import gsw as sw
import xarray as xr
import dask.array as dsa
import xgcm.grid
import netCDF4 as nc4
from numba import jit
import datetime
from datetime import datetime, date, time, timedelta

In [24]:
from dask.distributed import Client
client = Client()
client

/home/manjaree/miniconda3/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43147 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43147/status,
Dashboard: http://127.0.0.1:43147/status,Workers: 4
Total threads: 8,Total memory: 31.22 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42231,Workers: 4
Dashboard: http://127.0.0.1:43147/status,Total threads: 8
Started: Just now,Total memory: 31.22 GiB
Comm: tcp://127.0.0.1:36697,Total threads: 2
Dashboard: http://127.0.0.1:43499/status,Memory: 7.80 GiB
Nanny: tcp://127.0.0.1:44201,


In [25]:
#np.show_config()

In [26]:
pwd

'/home/manjaree/.ssh/oceanliner/testing'

In [27]:
cd /home/manjaree/.ssh/oceanliner/testing

/home/manjaree/.ssh/oceanliner/testing


In [28]:
#llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
#llc4320dir 

In [29]:
# --------------------------------------------------------------------
# USER INPUTS:
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 

# directory where model data is stored:
#llc4320dir = '/data1/adac/mitgcm/netcdf/'

#llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
#regiondir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
#regiondir = llc4320dir + RegionName + '/'
# directory to save derived data to - create if doesn't exist
#derivedir = regiondir + 'derived/'

#ocean computer
datadir = '/mnt/data/ACC_SMST/osse_model_input/30days/90days'  # input model data are here
outputdir = '/mnt/data/ACC_SMST/osse_output'   # interpolated data stored here
figdir = '/mnt/data/ACC_SMST/figures' # store figures#datadir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'  # input model data are here
regiondir = datadir
llc4320dir = datadir
#derivedir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/derived/'
derivedir = '/home/manjaree/.ssh/oceanliner/testing'
if not(os.path.isdir(derivedir)):
    os.mkdir(derivedir)

In [10]:
pwd

'/home/manjaree/.ssh/oceanliner/testing'

In [11]:
ls -l

total 7980
-rw-rw-r-- 1 manjaree manjaree    6320 Oct 20 13:56 argo_local.nc
-rw-rw-r-- 1 manjaree manjaree  217032 Oct 20 15:06 compute_model_derived_fields.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 dask-worker-space/
-rw-rw-r-- 1 manjaree manjaree  406468 Jul 14 08:46 demo.ipynb
-rw-rw-r-- 1 manjaree manjaree  853716 Aug 19 04:10 download_llc4320.ipynb
-rw-rw-r-- 1 manjaree manjaree    2389 Jul 28 11:57 example.ipynb
-rw-rw-r-- 1 manjaree manjaree  468806 Jul 20 17:08 final_project_updated.ipynb
-rw-rw-r-- 1 manjaree manjaree 4254898 Jul 20 17:06 get_swot_simulated_data.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 myData/
-rw-rw-r-- 1 manjaree manjaree   66882 Jul 14 08:46 osse_code_cloud.ipynb
-rw-rw-r-- 1 manjaree manjaree  504415 Jul 14 08:46 osse_code.ipynb
-rw-rw-r-- 1 manjaree manjaree   83441 Aug 16 11:20 OSSE_code_mb_10.ipynb
-rw-rw-r-- 1 manjaree manjaree  253053 Oct 20 14:46 OSSE_code_mb-Copy1.ipynb
-rw-rw-r-- 1 manjaree manjaree  665587 Aug 19 

In [12]:
# --------------------------------------------------------------------
# USER INPUTS:
# --------------------------------------------------------------------

# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
#RegionName = 'CapeBasin' 
RegionName = 'ACC_SMST'

# specify date range as start date & number of days.
start_date = date(2012,1,1)
# NOTE: ndays must be >1 
ndays = 10

# directory where data files are stored

# kd: lab
#datadir = '/Users/kdrushka/data/adac/mitgcm/netcdf/' + RegionName + '/'                      # input model data are here
#outputdir = '/Users/kdrushka/data/adac/osse_output/' + RegionName + '/'           # interpolated data stored here
#figdir = '/Users/kdrushka/Dropbox/projects/adac/figures/' + RegionName + '/' # store figures

#mb: laptop
#datadir = '/Volumes/TOSHIBA EXT 1/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input'  # input model data are here
#outputdir = '/Volumes/TOSHIBA EXT 1/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_output'   # interpolated data stored here
#figdir = '/Volumes/TOSHIBA EXT 1/LLC4320_pre-SWOT_10_days/ACC_SMST/figures' # store figures

#ocean computer
#datadir = '/mnt/data/CapeBasin/osse_model_input/30days'  # input model data are here
#outputdir = '/mnt/data/CapeBasin/osse_output'   # interpolated data stored here
#figdir = '/mnt/data/CapeBasin/figures' # store figures#datadir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'  # input model data are here

#ocean computer
datadir = '/mnt/data/ACC_SMST/osse_model_input/30days/90days'  # input model data are here
outputdir = '/mnt/data/ACC_SMST/osse_output'   # interpolated data stored here
figdir = '/mnt/data/ACC_SMST/figures' # store figures#datadir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'  # input model data are here


SAVE_FIGURES = False # True or False


# optional details for sampling (if not specified, reasonable defaults will be used)
# NOTE!! mooring and sim_mooring are different:
#    sim_mooring treats the mooring datapoints like a glider, 
#    whereas mooring interpolates directly to the mooring grid and should be faster
sampling_details_sim_glider = {
  'SAMPLING_STRATEGY' : 'sim_glider', 
#   'SAMPLING_STRATEGY' : 'trajectory_file', # options: sim_glider, sim_uctd, wave_glider or trajectory_file.add:  ASV
#   'SAMPLING_STRATEGY' : 'mooring', # options: sim_glider, sim_uctd, sim_mooring or trajectory_file.add: ASV. 
#    'SAMPLING_STRATEGY' : 'wave_glider', # options: wave_glider ..add trajectory file too?
    'PATTERN' : 'lawnmower', # back-forth or lawnmower 
   'zrange' : [-1, -1000],  # depth range of T/S profiles (down is negative). * add U/V range? *
#   'zmooring_TS' : list(range(-10,-1000,-10)) # instrument depths for moorings. T/S and U/V are the same.
 #   'zrange' : [-6, -100], # instrument depths for wave glider.  
  'hspeed' : 0.25,  # platform horizontal speed in m/s (for glider, uCTD)
 #   'hspeed' : 1,# platform horizontal (profile) speed in m/s  (for wave_glider)
   'vspeed' : 0.1, # platform vertical (profile) speed in m/s  (for glider, uCTD)
  # 'vspeed': 0 ,# platform vertical (profile) speed in m/s  (for wave_glider)
    'trajectory_file' : '../data/survey_trajectory_ACC_SMST_glider.nc', # if SAMPLING_STRATEGY = 'trajectory_file', specify trajectory file
    'AT_END' : 'reverse', # behaviour at and of trajectory: 'reverse', 'repeat' or 'terminate'. (could also 'restart'?)
    'DERIVED_VARIABLES' : False # specify whether or not to process the derived variables (steric height, rotated velocity, vorticity) - slower and takes significant to derive/save the stored variables
  }


sampling_details_mooring = {
#  'SAMPLING_STRATEGY' : 'sim_glider', 
#   'SAMPLING_STRATEGY' : 'trajectory_file', # options: sim_glider, sim_uctd, wave_glider or trajectory_file.add:  ASV
   'SAMPLING_STRATEGY' : 'mooring', # options: sim_glider, sim_uctd, sim_mooring or trajectory_file.add: ASV. 
#    'SAMPLING_STRATEGY' : 'wave_glider', # options: wave_glider ..add trajectory file too?
    'PATTERN' : 'lawnmower', # back-forth or lawnmower 
#   'zrange' : [-1, -1000],  # depth range of T/S profiles (down is negative). * add U/V range? *
  'zmooring_TS' : list(range(-10,-1000,-10)), # instrument depths for moorings. T/S and U/V are the same.
 #   'zrange' : [-6, -100], # instrument depths for wave glider.  
 # 'hspeed' : 0.25,  # platform horizontal speed in m/s (for glider, uCTD)
 #   'hspeed' : 1,# platform horizontal (profile) speed in m/s  (for wave_glider)
#   'vspeed' : 0.1, # platform vertical (profile) speed in m/s  (for glider, uCTD)
  # 'vspeed': 0 ,# platform vertical (profile) speed in m/s  (for wave_glider)
    'trajectory_file' : '../data/survey_trajectory_ACC_SMST_glider.nc', # if SAMPLING_STRATEGY = 'trajectory_file', specify trajectory file
#    'AT_END' : 'reverse', # behaviour at and of trajectory: 'reverse', 'repeat' or 'terminate'. (could also 'restart'?)
    'DERIVED_VARIABLES' : False, # specify whether or not to process the derived variables (steric height, rotated velocity, vorticity) - slower and takes significant to derive/save the stored variables
  }


sampling_details_wave_glider = {
  'SAMPLING_STRATEGY' : 'wave_glider', 
#   'SAMPLING_STRATEGY' : 'trajectory_file', # options: sim_glider, sim_uctd, wave_glider or trajectory_file.add:  ASV
#   'SAMPLING_STRATEGY' : 'mooring', # options: sim_glider, sim_uctd, sim_mooring or trajectory_file.add: ASV. 
 #   'SAMPLING_STRATEGY' : 'wave_glider', # options: wave_glider ..add trajectory file too?
    'PATTERN' : 'back-forth', # back-forth or lawnmower 
 #  'zrange' : [-1, -1000],  # depth range of T/S profiles (down is negative). * add U/V range? *
#   'zmooring_TS' : list(range(-10,-1000,-10)) # instrument depths for moorings. T/S and U/V are the same.
    'zrange' : [-6, -100], # instrument depths for wave glider.  
 # 'hspeed' : 0.25,  # platform horizontal speed in m/s (for glider, uCTD)
   'hspeed' : 1,# platform horizontal (profile) speed in m/s  (for wave_glider)
  # 'vspeed' : 0.1, # platform vertical (profile) speed in m/s  (for glider, uCTD)
   'vspeed': 0 ,# platform vertical (profile) speed in m/s  (for wave_glider)
   # 'trajectory_file' : '../data/survey_trajectory_ACC_SMST_glider.nc', # if SAMPLING_STRATEGY = 'trajectory_file', specify trajectory file
    'AT_END' : 'terminate', # behaviour at and of trajectory: 'reverse', 'repeat' or 'terminate'. (could also 'restart'?)
    'DERIVED_VARIABLES' : False # specify whether or not to process the derived variables (steric height, rotated velocity, vorticity) - slower and takes significant to derive/save the stored variables
}

In [13]:
regiondir

'/mnt/data/ACC_SMST/osse_model_input/30days/90days'

In [14]:
#os.listdir(regiondir) # returns list

In [15]:
#Checking each step for one single file
thisf = '/mnt/data/ACC_SMST/osse_model_input/30days/90days/LLC4320_pre-SWOT_ACC_SMST_20120207.nc'#file chosen at random
#print(thisf)
ds = xr.open_dataset(thisf)

# mean lat/lon of domain
xav = ds.XC.isel(j=0).mean(dim='i')
yav = ds.YC.isel(i=0).mean(dim='j')
print('center of domain: ', yav.values, ',' , xav.values)
ds

center of domain:  -55.282402 , 153.0


<xarray.Dataset>
Dimensions:   (j_g: 395, i: 480, i_g: 480, j: 395, k: 88, k_u: 88, k_l: 88,
               k_p1: 89, nb: 2, time: 24)
Coordinates:
  * j_g       (j_g) float32 0.0 1.0 2.0 3.0 4.0 ... 391.0 392.0 393.0 394.0
  * i         (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * i_g       (i_g) float32 0.0 1.0 2.0 3.0 4.0 ... 476.0 477.0 478.0 479.0
  * j         (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k         (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * k_u       (k_u) int32 0 1 2 3 4 5 6 7 8 9 ... 78 79 80 81 82 83 84 85 86 87
  * k_l       (k_l) int32 0 1 2 3 4 5 6 7 8 9 ... 78 79 80 81 82 83 84 85 86 87
  * k_p1      (k_p1) int32 0 1 2 3 4 5 6 7 8 9 ... 79 80 81 82 83 84 85 86 87 88
  * nb        (nb) int32 0 1
  * time      (time) datetime64[ns] 2012-02-07 ... 2012-02-07T23:00:00
Data variables: (12/35)
    XC        (j, i) float32 ...
    YC        (j, i) float32 ...
    DXV       (j, i) float32 ...
    DYU       (j, i) float32 ...
    Depth     (j, i) float32 ...
    AngleSN   (j, i) float32 ...
    ...        ...
    oceTAUY   (time, j_g, i) float64 ...
    Theta     (time, k, j, i) float64 ...
    Salt      (time, k, j, i) float64 ...
    U         (time, k, j, i_g) float32 ...
    V         (time, k, j_g, i) float64 ...
    W         (time, k_l, j, i) float64 ...
Attributes: (12/56)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Dimitris Menemenlis et al.
    contributor:                     Chris Hill, Christopher E. Henze, Jinbo ...
    contributor_role:                MITgcm developer, AMES supercomputer sup...
    cdm_data_type:                   Grid
    Conventions:                     CF-1.7, ACDD-1.3
    ...                              ...
    time_coverage_end:               2012-02-07 23:00:00
    time_coverage_start:             2012-02-07 00:00:00
    title:                           LLC4320 regional Southern Ocean
    geospatial_lon_resolution:       variable
    geospatial_lat_resolution:       variable
    platform:                        MITgcm

In [16]:
%%time
# for vorticity calculation, build the xgcm grid:
# see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                 'Y':{'center': 'j', 'left': 'j_g'},
                 'T':{'center': 'time'},
                 'Z':{'center': 'k'}})


CPU times: user 467 µs, sys: 53 µs, total: 520 µs
Wall time: 543 µs


In [17]:
print(grid)

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic, boundary=None):
  * center   j --> left
  * left     j_g --> center
T Axis (periodic, boundary=None):
  * center   time
Z Axis (periodic, boundary=None):
  * center   k


In [19]:

grid_ds=ds
u_transport = ds.UVEL * ds.dyG * ds.hFacW * ds.drF
v_transport = ds.VVEL * ds.dxG * ds.hFacS * ds.drF

AttributeError: 'Dataset' object has no attribute 'UVEL'

In [20]:
ds

<xarray.Dataset>
Dimensions:   (j_g: 395, i: 480, i_g: 480, j: 395, k: 88, k_u: 88, k_l: 88,
               k_p1: 89, nb: 2, time: 24)
Coordinates:
  * j_g       (j_g) float32 0.0 1.0 2.0 3.0 4.0 ... 391.0 392.0 393.0 394.0
  * i         (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * i_g       (i_g) float32 0.0 1.0 2.0 3.0 4.0 ... 476.0 477.0 478.0 479.0
  * j         (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k         (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * k_u       (k_u) int32 0 1 2 3 4 5 6 7 8 9 ... 78 79 80 81 82 83 84 85 86 87
  * k_l       (k_l) int32 0 1 2 3 4 5 6 7 8 9 ... 78 79 80 81 82 83 84 85 86 87
  * k_p1      (k_p1) int32 0 1 2 3 4 5 6 7 8 9 ... 79 80 81 82 83 84 85 86 87 88
  * nb        (nb) int32 0 1
  * time      (time) datetime64[ns] 2012-02-07 ... 2012-02-07T23:00:00
Data variables: (12/35)
    XC        (j, i) float32 ...
    YC        (j, i) float32 ...
    DXV       (j, i) float32 ...
    DYU       (j, i) float32 ...
    Depth     (j, i) float32 ...
    AngleSN   (j, i) float32 ...
    ...        ...
    oceTAUY   (time, j_g, i) float64 ...
    Theta     (time, k, j, i) float64 ...
    Salt      (time, k, j, i) float64 ...
    U         (time, k, j, i_g) float32 ...
    V         (time, k, j_g, i) float64 ...
    W         (time, k_l, j, i) float64 ...
Attributes: (12/56)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Dimitris Menemenlis et al.
    contributor:                     Chris Hill, Christopher E. Henze, Jinbo ...
    contributor_role:                MITgcm developer, AMES supercomputer sup...
    cdm_data_type:                   Grid
    Conventions:                     CF-1.7, ACDD-1.3
    ...                              ...
    time_coverage_end:               2012-02-07 23:00:00
    time_coverage_start:             2012-02-07 00:00:00
    title:                           LLC4320 regional Southern Ocean
    geospatial_lon_resolution:       variable
    geospatial_lat_resolution:       variable
    platform:                        MITgcm

In [24]:
%%time
# load reference file of argo data
# NOTE: could update to pull from ERDDAP or similar
#argoclimfile = '/data1/argo/argo_CLIM_3x3.nc'
argoclimfile = '/home/manjaree/.ssh/oceanliner/testing/argo_local.nc'
argods = xr.open_dataset(argoclimfile,decode_times=False) 
# reference profiles: annual average Argo T/S using nearest neighbor
Tref = argods["temp"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
Sref = argods["salt"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
# SA and CT from gsw:
# see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEV, yav)
Pref.compute()
SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                       dask='parallelized', output_dtypes=[Sref.dtype])
SAref.compute()
CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                       dask='parallelized', output_dtypes=[Sref.dtype])
CTref.compute()
Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                    dask='parallelized', output_dtypes=[Sref.dtype])
Dref.compute()
#print(Dref)

CPU times: user 15 ms, sys: 373 µs, total: 15.4 ms
Wall time: 15.3 ms


<xarray.DataArray (LEV: 27)>
array([1026.79572544, 1026.82071843, 1026.84481039, 1026.89948177,
       1026.9510571 , 1027.08425624, 1027.24825203, 1027.38602764,
       1027.54847285, 1027.70837588, 1028.08387161, 1028.44705226,
       1028.77603423, 1029.3517129 , 1029.90467858, 1030.42054904,
       1030.9234808 , 1031.4241963 , 1031.91925009, 1032.40953729,
       1032.89712342, 1033.3776902 , 1033.85898568, 1034.33487867,
       1034.8080404 , 1035.9911473 , 1037.17176102])
Coordinates:
  * LEV        (LEV) float64 0.0 5.0 10.0 20.0 ... 1.5e+03 1.75e+03 2e+03
    latitude   float64 -55.5
    longitude  float64 153.5
    i          float32 0.0
    j          float32 0.0

In [25]:
Dref

<xarray.DataArray (LEV: 27)>
array([1026.79572544, 1026.82071843, 1026.84481039, 1026.89948177,
       1026.9510571 , 1027.08425624, 1027.24825203, 1027.38602764,
       1027.54847285, 1027.70837588, 1028.08387161, 1028.44705226,
       1028.77603423, 1029.3517129 , 1029.90467858, 1030.42054904,
       1030.9234808 , 1031.4241963 , 1031.91925009, 1032.40953729,
       1032.89712342, 1033.3776902 , 1033.85898568, 1034.33487867,
       1034.8080404 , 1035.9911473 , 1037.17176102])
Coordinates:
  * LEV        (LEV) float64 0.0 5.0 10.0 20.0 ... 1.5e+03 1.75e+03 2e+03
    latitude   float64 -55.5
    longitude  float64 153.5
    i          float32 0.0
    j          float32 0.0

In [26]:
%%time
# create datasets for variables of interest:
ss = ds.Salt
tt = ds.Theta
pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

CPU times: user 1 s, sys: 41.7 ms, total: 1.04 s
Wall time: 1 s


In [ ]:
%%time
# 1. compute absolute salinity and conservative temperature
sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, output_dtypes=[ss.dtype])
sa.compute()
ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, output_dtypes=[ss.dtype])
ct.compute()
dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, output_dtypes=[ss.dtype])
dd.compute()

In [107]:
%%time
# 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, output_dtypes=[ss.dtype])
sva.compute()

NameError: name 'sa' is not defined

In [16]:
%%time
# 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
# - first, interpolate Dref to the model pressure levels
Drefi = Dref.interp(LEV=-ds.Z)
dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

# steric height computation (summation/integral)
# - increase the size of Drefi and dz to match the size of sva
Db = Drefi.broadcast_like(sva)
dzb = dz.broadcast_like(sva)
dum = Db * sva * dzb
sh = dum.cumsum(dim='k')

CPU times: user 3.16 s, sys: 1.03 s, total: 4.19 s
Wall time: 4.19 s


In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
#vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')


In [ ]:
vorticity

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'X', boundary='extend')

In [ ]:
vorticity

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'Y', boundary='extend')


In [24]:
fis = range(len(fg))
fis

range(0, 10)

In [27]:
yn = 'y'
yn

'y'

In [ ]:
def compute_derived_fields(RegionName, datadir, start_date, ndays):
    """
    Check for derived files in {datadir}/derived and compute if the files don't exist
    """
    # directory to save derived data to - create if doesn't exist
    derivedir = datadir + 'derived/'
    if not(os.path.isdir(derivedir)):
        os.mkdir(derivedir)
        
    # files to load:
    date_list = [start_date + datetime.timedelta(days=x) for x in range(ndays)]
    target_files = [f'{datadir}LLC4320_pre-SWOT_{RegionName}_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files
    
    # list of derived files:
    derived_files = [f'{derivedir}LLC4320_pre-SWOT_{RegionName}_derived-fields_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files

        
    # loop through input files, then do the following:
    # - compute steric height
    # - interpolate vector quantities (velocity and wind) to the tracer grid
    # - rotate vectoor quantities to the geophysical (east/north) grid 
    # - compute vorticity (on the transformed grid)
    fis = range(len(target_files))
    
    cnt = 0 # count
    for fi in fis:
        # input filename:
        thisf=target_files[fi]
        # output filename:
        fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
        fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
        # check if output file already exists
        if (not(os.path.isfile(fnout))):   
            print('computing derived fields for', thisf) 
            # load file:
            ds = xr.open_dataset(thisf)
            
            # -------
            # first time through the loop, load reference profile:
            # load a single file to get coordinates
            if cnt==0:
                # mean lat/lon of domain
                xav = ds.XC.isel(j=0).mean(dim='i')
                yav = ds.YC.isel(i=0).mean(dim='j')

                # for transforming U and V, and for the vorticity calculation, build the xgcm grid:
                # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
                grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                             'Y':{'center': 'j', 'left': 'j_g'},
                             'T':{'center': 'time'},
                             'Z':{'center': 'k'}})
                

                # --- load reference file of argo data
                # here we use the 3x3 annual mean Argo product on standard produced by IRPC & distributed by ERDDAP
                # https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_defb_b79c_cb17.html
                # - download the profile closest to xav,yav once (quick), use it, then delete it.
                
                # URL gets temp & salt at all levels
                argofile = f'https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_625d_3b64_cc4d.nc?temp[(0000-12-15T00:00:00Z):1:(0000-12-15T00:00:00Z)][(0.0):1:(2000.0)][({yav.data}):1:({yav.data})][({xav.data}):1:({xav.data})],salt[(0000-12-15T00:00:00Z):1:(0000-12-15T00:00:00Z)][(0.0):1:(2000.0)][({yav.data}):1:({yav.data})][({xav.data}):1:({xav.data})]'
                
                # delete the argo file if it exists 
                if os.path.isfile('argo_local.nc'):
                    os.remove('argo_local.nc')
                # use requests to get the file, and write locally:
                r = requests.get(argofile)
                file = open('argo_local.nc','wb')
                file.write(r.content)
                file.close()
                # open the argo file:
                argods = xr.open_dataset('argo_local.nc',decode_times=False)
                # get rid of time coord/dim/variable, which screws up the time in ds if it's loaded
                argods = argods.squeeze().reset_coords(names = {'time'}, drop=True) 
                # reference profiles: annual average Argo T/S using nearest neighbor
                Tref = argods["temp"]
                Sref = argods["salt"]
                # SA and CT from gsw:
                # see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
                Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEV, yav)
                Pref.compute()
                SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                                        output_dtypes=[Sref.dtype])
                SAref.compute()
                CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                                        output_dtypes=[Sref.dtype])
                CTref.compute()
                Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                                      output_dtypes=[Sref.dtype])
                Dref.compute()
                
                
                cnt = cnt+1
                print()
                
            # -------
            
            # --- COMPUTE STERIC HEIGHT IN STEPS ---
            # 0. create datasets for variables of interest:
            ss = ds.Salt
            tt = ds.Theta
            pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))
            
            # 1. compute absolute salinity and conservative temperature
            sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
            sa.compute()
            ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
            ct.compute()
            dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
            dd.compute()
            # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
            sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
            sva.compute()
            # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
            # - first, interpolate Dref to the model pressure levels
            Drefi = Dref.interp(LEV=-ds.Z)
            dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

            # steric height computation (summation/integral)
            # - increase the size of Drefi and dz to match the size of sva
            Db = Drefi.broadcast_like(sva)
            dzb = dz.broadcast_like(sva)
            dum = Db * sva * dzb
            sh = dum.cumsum(dim='k') 
            # this gives sh as a 3-d variable, (where the depth dimension 
            # represents the deepest level from which the specific volume anomaly was interpolated)
            # - but in reality we just want the SH that was determined by integrating over
            # the full survey depth, which gives a 2-d output:
            sh_true = dum.sum(dim='k') 
            
            # --- COMPUTE VORTICITY using xgcm and interpolate to X, Y
            # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
            vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
            vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')
            
            

            # --- ROTATE AND TRANSFORM VECTOR QUANTITIES ---
            # interpolate U,V and oceTAUX, oceTAUY to the tracer grid
            # and rotate them to geophysical (east, north) coordinates instead of model ones:
            # 1) regrid 
            print('interpolating to tracer grid')
            U_c = grid.interp(ds.U, 'X', boundary='extend')
            V_c = grid.interp(ds.V, 'Y', boundary='extend')
            # do the same for TAUX and TAUY:
            oceTAUX_c = grid.interp(ds.oceTAUX, 'X', boundary='extend')
            oceTAUY_c = grid.interp(ds.oceTAUY, 'Y', boundary='extend')

            # 2) rotate U and V, and taux and tauy, using rotate_vector_to_EN:
            print('rotating to east/north')
            U_transformed, V_transformed = rotate_vector_to_EN(U_c, V_c, ds['AngleCS'], ds['AngleSN'])
            oceTAUX_transformed, oceTAUY_transformed = rotate_vector_to_EN(oceTAUX_c, oceTAUY_c, ds['AngleCS'], ds['AngleSN'])

            # --- save derived fields in a new file
            # - convert sh and zeta to datasets
            # NOTE can do this more efficiently in a single line w/out converting to dataset???
            dout = vorticity.to_dataset(name='vorticity')
            sh_ds = sh.to_dataset(name='steric_height')
            dout = dout.merge(sh_ds)
            sh_true_ds = sh_true.to_dataset(name='steric_height_true')
            dout = dout.merge(sh_true_ds)
            U_transformed_ds = U_transformed.to_dataset(name='U_transformed')
            V_transformed_ds = V_transformed.to_dataset(name='V_transformed')
            oceTAUX_transformed_ds = oceTAUX_transformed.to_dataset(name='oceTAUX_transformed')
            oceTAUY_transformed_ds = oceTAUY_transformed.to_dataset(name='oceTAUY_transformed')
            dout = dout.merge(U_transformed_ds).merge(V_transformed_ds)
            dout = dout.merge(oceTAUX_transformed_ds).merge(oceTAUY_transformed_ds)
            
            
            
            # add/rename the Argo reference profile variables to dout:
            tref = Tref.to_dataset(name='Tref')
            tref = tref.merge(Sref).rename({'salt': 'Sref'}).\
                rename({'LEV':'zref','latitude':'yav','longitude':'xav'})
            # - add ref profiles to dout and drop uneeded vars/coords
            dout = dout.merge(tref).drop_vars({'longitude','latitude','LEV'})
  
    
            # - add attributes for all variables
            dout.steric_height.attrs = {'long_name' : 'Steric height',
                                    'units' : 'm',
                                    'comments_1' : 'Computed by integrating the specific volume anomaly (SVA) multiplied by a reference density, where the reference density profile is calculated from temperature & salinity profiles from the APDRC 3x3deg gridded Argo climatology product (accessed through ERDDAP). The profile nearest to the center of the domain is selected, and T & S profiles are averaged over one year before computing ref density. SVA is computed from the model T & S profiles. the Gibbs Seawater Toolbox is used compute reference density and SVA. steric_height is given at all depth levels (dep): steric_height at a given depth represents steric height signal generated by the water column above that depth - so the deepest steric_height value represents total steric height (and is saved in steric_height_true'
                                       }
            dout.steric_height_true.attrs = dout.steric_height.attrs
            
            dout.vorticity.attrs = {'long_name' : 'Vertical component of the vorticity',
                                    'units' : 's-1',
                                    'comments_1' : 'computed on DXG,DYG then interpolated to X,Y'}
            
            dout.U_transformed.attrs['long_name'] = "Horizontal velocity in the eastward direction"
            dout.U_transformed.attrs['comments_1'] = "Horizontal velocity in the eastward direction at the center of the tracer cell on the native model grid."
            dout.U_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.V_transformed.attrs['long_name'] = "Horizontal velocity in the northward direction"
            dout.V_transformed.attrs['comments_1'] = "Horizontal velocity in the northward direction at the center of the tracer cell on the native model grid."
            dout.V_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.oceTAUX_transformed.attrs['long_name'] = "Ocean surface stress in the eastward direction"
            dout.oceTAUX_transformed.attrs['comments_1'] = "Ocean surface stress due to wind and sea-ice in the eastward direction centered over the the native model grid"
            dout.oceTAUX_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.oceTAUY_transformed.attrs['long_name'] = "Ocean surface stress in the northward direction"
            dout.oceTAUY_transformed.attrs['comments_1'] = "Ocean surface stress due to wind and sea-ice in the northward direction centered over the the native model grid"
            dout.oceTAUY_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."
            
            
            
            dout.Tref.attrs = {'long_name' : f'Reference temperature profile at {yav.data}N,{xav.data}E',
                                    'units' : 'degree_C',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            dout.Sref.attrs = {'long_name' : f'Reference salinity profile at {yav.data}N,{xav.data}E',
                                    'units' : 'psu',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            
            dout.zref.attrs = {'long_name' : f'Reference depth for Tref and Sref',
                                    'units' : 'm',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            
            
            # - save netcdf file with derived fields
            netcdf_fill_value = nc4.default_fillvals['f4']
            dv_encoding = {}
            for dv in dout.data_vars:
                dv_encoding[dv]={'zlib':True,  # turns compression on\
                            'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                            'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                            'dtype':'float32',\
                            '_FillValue':netcdf_fill_value}
            # save to a new file
            print(' ... saving to ', fnout)
            # TROUBLESHOOTING::::: DELETE THE RETURN LINE
            #return dout, dv_encoding
            dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)

            
            
            # release & delete Argo file
            argods.close()
#             os.remove('argo_local.nc')

In [30]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# loop through files, then compute steric height, vorticity, etc. on the i/j grid
# fis = range(1)
#fis = range(len(fg))

for fi in range(len(fg)):
    # --- select data ---
    thisf=fg[fi]
    
    # check if output file already exists
    fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
    fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
    
    if (os.path.isfile(fnout) & (yn.lower() == 'y')):
        yn = input(f'\n{fnout} already exists. Overwrite? (this decision will apply to all files) [y/N]')
    if (os.path.isfile(fnout) & (yn.lower() == 'n')):
        # do nothing
        1
    elif (yn.lower() == 'y') | (not(os.path.isfile(fnout))):   
        print(thisf , '(' , fi+1, 'of', len(fis), ')')  
        ds = xr.open_dataset(thisf)

        # create datasets for variables of interest:
        ss = ds.Salt
        tt = ds.Theta
        pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

        # --- compute steric height in steps ---
        # 1. compute absolute salinity and conservative temperature
        sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
        sa.compute()
        ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
        ct.compute()
        dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        dd.compute()
        # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
        sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        sva.compute()
        # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
        # - first, interpolate Dref to the model pressure levels
        Drefi = Dref.interp(LEV=-ds.Z)
        dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

        # steric height computation (summation/integral)
        # - increase the size of Drefi and dz to match the size of sva
        Db = Drefi.broadcast_like(sva)
        dzb = dz.broadcast_like(sva)
        dum = Db * sva * dzb
        sh = dum.cumsum(dim='k')

        # --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
        #vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
        #vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

        # --- save derived fields in a new file
        # - convert sh and zeta to datasets
        #dout = vorticity.to_dataset(name='vorticity')
        sh_ds = sh.to_dataset(name='steric_height')
        dout = dout.merge(sh_ds)
        # add/rename the Argo reference profile variables
        tref = Tref.to_dataset(name='Tref')
        tref = tref.merge(Sref).rename({'SALT': 'Sref'}).\
            rename({'LEV':'zref','latitude':'yav','longitude':'xav'}).\
            drop_vars({'i','j'})
        # - add ref profiles to dout and drop uneeded vars/coords
        dout = dout.merge(tref).drop_vars({'longitude','latitude','LEV','i','j'})

        # - save netcdf file with derived fields
        netcdf_fill_value = nc4.default_fillvals['f4']
        dv_encoding = {}
        for dv in dout.data_vars:
            dv_encoding[dv]={'zlib':True,  # turns compression on\
                        'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                        'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                        'dtype':'float32',\
                        '_FillValue':netcdf_fill_value}
        # save to a new file
        print(' ... saving to ', fnout)
        dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)


SyntaxError: invalid syntax (<ipython-input-30-69d7b57ebbd2>, line 6)